## kosis
- 업종에 대한 실적 정보를 제공하는 사이트

In [1]:
from selenium import webdriver
from fake_useragent import UserAgent
import time

In [18]:
def crawling(start, end, headless=True): 
    
    def loading(driver, term=1, maximum=10):
        """
        wating for data loading time
        parameter : term : check loaded term (sec)
                  : maximum : maximum check loaded term (sec)
        return : -
        """
        print("loading.", end="")
        while True:
            style = driver.find_element_by_css_selector("#disPlayBox").get_attribute("style")
            time.sleep(term)
            
            # 로딩이 완료되었는지 확인             
            if style == "display: none;":
                print("done!")
                break
            
            # 최대 기다리는 시간 체크         
            maximum -= 1
            if not maximum:
                break
            
            print(".", end="")
                    
    # driver option 설정
    options = webdriver.ChromeOptions()
    
    # user agent 설정
    options.add_argument("user-agent={}".format(UserAgent().chrome))
    
    # headless 설정
    if headless:
        options.add_argument('headless')
    
    # 옵션 적용
    driver = webdriver.Chrome(options=options)
    
    # 웹 페이지 이동
    driver.get("http://kosis.kr/statHtml/statHtml.do?orgId=301&tblId=DT_041Y013&conn_path=I2")
    
    # 로딩 체크
    loading(driver)
    
    # rows
    rows = driver.find_elements_by_css_selector("#mainTable > tbody > tr")
    
    # 날짜 설정
    driver.find_element_by_css_selector("#tabTimeText").click()
    driver.find_element_by_css_selector(\
        "#timeM > h2.top > select:first-child > option[value='{}']".format(start)).click()
    driver.find_element_by_css_selector(\
        "#timeM > h2.top > select:nth-child(2) > option[value='{}']".format(end)).click()
    driver.execute_script("fn_search();")
    
    # 로딩 체크
    loading(driver)
    
    # columns 
    columns = driver.find_elements_by_css_selector("#mainTableT > thead > tr:first-child > th")[2:]
    columns = [column.text for column in columns]
    
    # rows
    rows = driver.find_elements_by_css_selector("#mainTable > tbody > tr")
    rows_count = len(rows)
    
    indexs, values, tmp_index = [],  [], ""
    for count, row in enumerate(rows):
        
        # 마지막 줄이 아니면        
        if count != (rows_count - 1):
            row_indexs = row.find_elements_by_css_selector(".first")
        # 마지막줄 이면
        else:
            row_indexs = row.find_elements_by_css_selector(".first-end")
        
        # indexs 조합         
        if len(row_indexs) > 1:
            tmp_index = row_indexs[0].text
            indexs.append("{}:{}".format(tmp_index, row_indexs[1].text))
        else:
            indexs.append("{}:{}".format(tmp_index, row_indexs[0].text))
        
        # values 가져오기
        row_values = row.find_elements_by_css_selector(".value > .val")
        row_values = [value.text for value in row_values]
        values.append(row_values)
    
    # 브라우져 종료
    driver.quit()
    
    # 데이터 프레임으로 만들기
    return pd.DataFrame(values, columns=columns, index=indexs)
    
    
# test code    
df = crawling("201803", "201806", headless=True)
df.tail()

loading..done!
loading..done!


2018. 06  2018. 05  2018. 04  2018. 03
  서비스업: 업황실적          80        85        81        80
  서비스업: 매출실적          87        88        86        87
  서비스업: 채산성실적         89        90        88        87
  서비스업: 자금사정실적        87        89        86        85
  서비스업: 인력사정실적        80        81        86        86

In [19]:
df.T

전 산 업: 업황실적  전 산 업: 매출실적  전 산 업: 채산성실적  전 산 업: 자금사정실적  \
 2018. 06           80           87            87             86   
 2018. 05           81           87            87             86   
 2018. 04           79           84            86             84   
 2018. 03           77           82            86             83   

           전 산 업: 인력사정실적   제 조 업: 업황실적   제 조 업: 업황실적(S.A)   제 조 업: 매출실적  \
 2018. 06             86            80                 79            88   
 2018. 05             86            78                 74            86   
 2018. 04             92            77                 72            82   
 2018. 03             92            74                 72            78   

            제 조 업: 수출실적   제 조 업: 내수판매실적      ...         비제조업: 업황실적(S.A)  \
 2018. 06            91              85      ...                      79   
 2018. 05            87              85      ...                      80   
 2018. 04            86              81      ...                      77   
 2018. 03            81              76      ...                      78   

            비제조업: 매출실적   비제조업: 채산성실적   비제조업: 자금사정실적   비제조업: 인력사정실적  \
 2018. 06           87            88             87             81   
 2018. 05           87            89             88             82   
 2018. 04           85            87             86             87   
 2018. 03           86            86             86             87   

            서비스업: 업황실적   서비스업: 매출실적   서비스업: 채산성실적   서비스업: 자금사정실적  \
 2018. 06           80           87            89             87   
 2018. 05           85           88            90             89   
 2018. 04           81           86            88             86   
 2018. 03           80           87            87             85   

            서비스업: 인력사정실적  
 2018. 06             80  
 2018. 05             81  
 2018. 04             86  
 2018. 03             86  

[4 rows x 122 columns]

### Class

In [34]:
from selenium import webdriver
from fake_useragent import UserAgent
import pandas as pd
import time

class Kosis:
    
    def __init__(self, start_date=None, end_date=None, headless=True, url="http://kosis.kr/statHtml/statHtml.do?orgId=301&tblId=DT_041Y013&conn_path=I2"):
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent={}".format(UserAgent().chrome))
        if headless:
            options.add_argument('headless')
        self.driver = webdriver.Chrome(options=options)
        self.start_date = start_date
        self.end_date = end_date
        self.url = url
        self.rows = None
        self.columns = None
        self.values = None
        self.result_df = None
        
    def __loading(self, term=1, maximum=10):
        print("loading.", end="")
        while True:
            style = self.driver.find_element_by_css_selector("#disPlayBox").get_attribute("style")
            time.sleep(term)
            if style == "display: none;":
                print("done!")
                break
            maximum -= 1
            if not maximum:
                break
            
            print(".", end="")
    
    def __select_date(self):
        self.driver.find_element_by_css_selector("#tabTimeText").click()
        self.driver.find_element_by_css_selector(\
            "#timeM > h2.top > select:first-child > option[value='{}']".format(self.start_date)).click()
        self.driver.find_element_by_css_selector(\
            "#timeM > h2.top > select:nth-child(2) > option[value='{}']".format(self.end_date)).click()
        self.driver.execute_script("fn_search();")

    def __rows(self):
        return self.driver.find_elements_by_css_selector("#mainTable > tbody > tr")
    
    def __columns(self):
        columns = self.driver.find_elements_by_css_selector("#mainTableT > thead > tr:first-child > th")[2:]
        return [column.text for column in columns]
    
    def __indexs(self):
        indexs, tmp_index = [], ""
        rows_count = len(self.rows)
        for count, row in enumerate(self.rows):
            
            if count != (rows_count - 1):
                row_indexs = row.find_elements_by_css_selector(".first")
            else:
                row_indexs = row.find_elements_by_css_selector(".first-end")

            if len(row_indexs) > 1:
                tmp_index = row_indexs[0].text
                indexs.append("{}:{}".format(tmp_index, row_indexs[1].text))
            else:
                indexs.append("{}:{}".format(tmp_index, row_indexs[0].text))
                
        return indexs

    def __values(self):
        values = []
        for count, row in enumerate(self.rows):
            row_values = row.find_elements_by_css_selector(".value > .val")
            row_values = [value.text for value in row_values]
            values.append(row_values)
        return values
    
    def crawling(self):
        
        self.driver.get(self.url)
        self.__loading()
        
        if self.start_date and self.end_date:
            print("set date...")
            self.__select_date()
            
        self.__loading()
        
        print("crawling...", end="")
        self.rows = self.__rows()
        self.columns = self.__columns()
        self.indexs = self.__indexs()
        self.values = self.__values()        
        print("done!")
        
        self.result_df = pd.DataFrame(self.values, columns=self.columns, index=self.indexs)
        return self.result_df
    
    def set_date(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        
    def close(self):
        self.driver.quit()
        
# test code
kosis = Kosis("201803", "201807")
df = kosis.crawling()
df.tail()

loading...done!
set date...
loading...done!
crawling...


2018. 07  2018. 06  2018. 05  2018. 04  2018. 03
  서비스업: 업황실적          75        80        85        81        80
  서비스업: 매출실적          83        87        88        86        87
  서비스업: 채산성실적         86        89        90        88        87
  서비스업: 자금사정실적        87        87        89        86        85
  서비스업: 인력사정실적        82        80        81        86        86

In [28]:
kosis.close()